In [ ]:
from google.colab import drive

In [ ]:
import os

In [13]:
print(os.getcwd())

/content


In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_dir = '/content/drive/My Drive/CH Counts Data'

In [ ]:

#Lists all files up to number specified in the google drive folder '/content/drive/My Drive/CH Counts Data'
train_file_numbers = range(1, 1000)
train_file_names = [os.path.join(data_dir, f"Group8_{str(i).zfill(4)}.csv") for i in train_file_numbers]

train_file_numbers = range(1, 1000)
train_file_all_names = [os.path.join(data_dir, f"Group8_{str(i).zfill(4)}.csv") for i in train_file_numbers]
train_file_names


In [ ]:
import pandas as pd


unique_rows = []
seen_classes = set()

count = 0

#Grabs one row very every unique class seen, is needed for warm start RF
for name in train_file_all_names:
      print(count/len(train_file_all_names)*100)
      count +=1
      df = pd.read_csv(name)
      df = df.fillna(0)
      unique_values_in_df = set(df['Mineral Phase'].unique()) - seen_classes
      seen_classes.update(unique_values_in_df)
      for unique_value in unique_values_in_df:
          filtered_df = df[df['Mineral Phase'] == unique_value]
          if not filtered_df.empty:
              unique_row = filtered_df.iloc[0]
              unique_rows.append(unique_row)

#Each unique classes row is added to a df
unique_df = pd.DataFrame(unique_rows)
unique_df = unique_df.drop(unique_df.columns[:126], axis=1)

0.0
0.10010010010010009
0.20020020020020018
0.3003003003003003
0.40040040040040037
0.5005005005005005
0.6006006006006006
0.7007007007007007
0.8008008008008007
0.9009009009009009
1.001001001001001
1.1011011011011012
1.2012012012012012
1.3013013013013013
1.4014014014014013
1.5015015015015014
1.6016016016016015
1.7017017017017018
1.8018018018018018
1.9019019019019021
2.002002002002002
2.1021021021021022
2.2022022022022023
2.3023023023023024
2.4024024024024024
2.5025025025025025
2.6026026026026026
2.7027027027027026
2.8028028028028027
2.902902902902903
3.003003003003003
3.1031031031031033
3.203203203203203
3.303303303303303
3.4034034034034035
3.5035035035035036
3.6036036036036037
3.7037037037037033
3.8038038038038042
3.903903903903904
4.004004004004004
4.1041041041041035
4.2042042042042045
4.3043043043043046
4.404404404404405
4.504504504504505
4.604604604604605
4.704704704704705
4.804804804804805
4.904904904904905
5.005005005005005
5.105105105105105
5.205205205205205
5.305305305305305
5.40

In [ ]:
len(unique_df)

30

In [ ]:
print(unique_df["Mineral Phase"])

20           Fe-Silicate
2167      Fe-Sulphide_Zn
1085    AlFe-Silicate_Zn
73              Ilmenite
1166              Quartz
8             Background
345             Zn-Oxide
6986              Rutile
0               Fe-Oxide
339        AlFe-Silicate
4492             Unknown
45          AlK-Silicate
724             Monazite
5078           Leucoxene
416        AlMn-Silicate
1086            Xenotime
162               Barite
3502            Mn-Oxide
262               Zircon
2201           Celestine
7386         Al-Silicate
6619           Amphibole
6898             Epidote
2753             Apatite
3665            Pb-Oxide
3969            Cu-Oxide
3132      Fe-sulphide_Pb
6748            Titanite
6807            Al-Oxide
197          Fe-Sulphide
5207       AlCa-Silicate
3862             Thorite
1787        REE-Silicate
493              RM ONLY
2148           Carbonate
2853    Nb-bearing phase
5759         Cassiterite
4384            Chromite
5054                   0
4630        Columbite_Mn


In [ ]:
from sklearn.ensemble import RandomForestClassifier
#Warm start RF model, paramater tuning needed
"""forest_classifier = RandomForestClassifier(
    warm_start = True,
    n_estimators=1,
    max_depth=None,
    bootstrap=True,
    class_weight="balanced"
)"""

count = 0


In [ ]:
from sklearn.metrics import accuracy_score
from joblib import load, dump

test_file_name = "/content/drive/My Drive/CH Counts Data/Group8_2000.csv"

test_df = pd.read_csv(test_file_name)
test_df = test_df.drop(test_df.columns[:126], axis=1)
test_df = pd.concat([test_df,unique_df], ignore_index = True)
test_df = test_df.fillna(0)

X_test = test_df.iloc[:,:-1]
y_test = test_df.iloc[:,-1].astype(str)


#Adding each file as unique estimator or tree, needs to see all classes first
for  file_name in train_file_names:
    print(count/len(train_file_names))
    count += 1
    if count%50 == 0:
      dump(forest_classifier,'forest_classifier.pkl')
      y_pred = forest_classifier.predict(X_test)
      accuracy = accuracy_score(y_test, y_pred)
      print(f"Accuracy: {accuracy * 100:.2f}%")
    df = pd.read_csv(file_name)
    df = df.drop(df.columns[:126], axis=1)
    df = pd.concat([unique_df,df], ignore_index = True)
    df = df.fillna(0)
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1].astype(str)
    forest_classifier.fit(X, y)
    forest_classifier.n_estimators += 1




0.0


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:777: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight ("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the resulting weights as the class_weight parameter.
  warn(


0.001001001001001001


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:777: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight ("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the resulting weights as the class_weight parameter.
  warn(


0.002002002002002002


...

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:777: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight ("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the resulting weights as the class_weight parameter.
  warn(


0.997997997997998


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:777: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight ("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the resulting weights as the class_weight parameter.
  warn(


0.998998998998999


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:777: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight ("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the resulting weights as the class_weight parameter.
  warn(


In [ ]:
#Classification Report

#Test file
test_file_name = "/content/drive/My Drive/CH Counts Data/Group8_1090.csv"
test_df = pd.read_csv(test_file_name)
test_df = test_df.drop(test_df.columns[:126], axis=1)
X_test = test_df.iloc[:,:-1]
y_test = test_df.iloc[:,-1].astype(str)



y_pred = forest_classifier.predict(X_test)

unique_classes = y_test.unique().tolist()
from sklearn.metrics import classification_report
report = classification_report(y_test, y_pred, target_names=unique_classes)
print(report)

                  precision    recall  f1-score   support

          Rutile       0.00      0.00      0.00         1
        Fe-Oxide       0.00      0.00      0.00         1
     Fe-Silicate       0.00      0.00      0.00         1
   AlFe-Silicate       0.65      0.08      0.14       374
      Background       0.00      0.00      0.00         1
       Amphibole       0.81      0.32      0.46        78
        Ilmenite       0.85      0.84      0.84       113
          Zircon       0.93      0.87      0.90      2610
   AlMn-Silicate       0.83      1.00      0.90      1329
         Epidote       1.00      0.78      0.87        94
     Cassiterite       0.00      0.00      0.00         1
       Leucoxene       0.86      0.51      0.64       389
         Unknown       0.56      0.94      0.70       878
        Titanite       0.82      0.25      0.38       181
          Quartz       0.00      0.00      0.00         9
    AlK-Silicate       0.90      0.96      0.93       990
        Cu-Ox

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import pandas as pd
from joblib import load, dump
from sklearn.metrics import accuracy_score
forest_classifier = load("forest_classifier.pkl")
#Finding out the average accuracy over files specified
validation_file_numbers = range(1050, 1060)
validation_file_all_names = [os.path.join(data_dir, f"Group8_{str(i).zfill(4)}.csv") for i in validation_file_numbers]

accuracy_list = []
count = 0
for test_file in validation_file_all_names:
  print(count)
  count +=1
  test_df = pd.read_csv(test_file)
  test_df = test_df.drop(test_df.columns[:126], axis=1)
  test_df = test_df.fillna(0)
  X_test = test_df.iloc[:,:-1]
  y_test = test_df.iloc[:,-1].astype(str)
  y_pred = forest_classifier.predict(X_test)
  accuracy = accuracy_score(y_test, y_pred)
  accuracy_list.append(accuracy)

sum(accuracy_list)/len(accuracy_list)

FileNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

# Old RF trained on first 20 datasets
test_file_name = "/content/drive/My Drive/CH Counts Data/Group8_0026.csv"
test_df = pd.read_csv(test_file_name)
test_df = test_df.drop(test_df.columns[:126], axis=1)

test_df = pd.concat([test_df,unique_df], ignore_index = True)
test_df = test_df.fillna(0)



X_test = test_df.iloc[:,:-1]
y_test = test_df.iloc[:,-1].astype(str)
print(len(y_test.unique()))


y_pred = forest_classifier.predict(X_test)


from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy * 100:.2f}%")


31
Accuracy: 91.68%


In [ ]:
for index, estimator in enumerate(forest_classifier.estimators_):
    y_pred_train_tree = estimator.predict(X)
    unique_y_pred_train_tree = set(y_pred_train_tree)
    if len(unique_y_pred_train_tree) != 49:
        print(f"Tree {index} recognizes {len(unique_y_pred_train_tree)} classes.")

Unique classes in y_test:
49


In [ ]:
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import numpy as np
label_encode = LabelEncoder()
xgb_model = None

params = {
    'objective': 'multi:softmax',  # Objective for multi-class classification
    'eta': 2,   # Learning rate
    'max_depth': 6
}

label_encode.fit(list(seen_classes))

for i,name in enumerate(train_file_names):
  df = pd.read_csv(name)
  df = df.drop(df.columns[:126], axis=1)
  if i == 0:
      feature_names = df.columns.tolist()[:-10]
  X = df.iloc[:,:-10]
  y = df.iloc[:,-1]
  y = label_encode.transform(y)
  num_classes = len(seen_classes)
  dtrain = xgb.DMatrix(X,label = y,feature_names=feature_names)
  params['num_class'] = num_classes
  xgb_model = xgb.train(params,dtrain,xgb_model=xgb_model)
  print(f"Batch {i+1} training completed.")


xgb_model.save_model("xgboost_model1.model")



Batch 1 training completed.
Batch 2 training completed.
Batch 3 training completed.


ValueError: ignored

In [ ]:
train_file_numbers = range(80, 100)
train_file_all_names = [os.path.join(data_dir, f"Group8_{str(i).zfill(4)}.csv") for i in train_file_numbers]
train_file_names



for name in validation_file_name:
  df = pd.read_csv(name)
  df = df.drop(df.columns[:126], axis = 1)
  X_val = df.iloc[:,:-1]
  y_val = df.iloc[:,-1]
  dval = xgb.Dmatrix(X_val,label =y_val)



y_pred = xgb_model.predict(dval)

accuracy = accuracy_score(y_val,y_pred)

print(f"Accuracy: {accuracy * 100:.2f}%")


In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_val, y_pred.round())


TypeError: ignored

In [ ]:
unique_df.head()

,CH_124,CH_125,CH_126,CH_127,CH_128,CH_129,CH_130,CH_131,CH_132,CH_133,...,CH_4089,CH_4090,CH_4091,CH_4092,CH_4093,CH_4094,CH_4095,CH_4096,Sum,Mineral Phase
20,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0.0,9072,Fe-Silicate
2167,0,0,0,0,0,1,0,0,1,0,...,1,0,0,0,1,1,1,1.0,3883,Fe-Sulphide_Zn
1085,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,1,1,1,1.0,4704,AlFe-Silicate_Zn
73,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0.0,8369,Ilmenite
1166,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,1,1.0,6478,Quartz
